<a href="https://colab.research.google.com/github/lawho13/Structural-analysis-of-regional-economies-based-on-I-O-data/blob/main/notebooks/creatin_dict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
# from matplotlib.lines import Line2D


from google.colab import drive
drive.mount('/content/drive')

#data: https://www.rug.nl/ggdc/valuechain/long-run-wiod
#clicked "WIOT Tables Excel"
#size is 168mb qualifying for all 30 bonus points

#need to upload the csv to your google drive to read

df = pd.read_csv("/content/drive/My Drive/CS439 Final Project/lr_wiod_wiot_wide.csv")


Mounted at /content/drive


In [ ]:
#countries in dataset
# print(df['row_country'].value_counts(dropna=False))

#codes for sectors i presume

df.rename(columns={"year": "Year", "row_isic3": "Sector"}, inplace=True)

code_to_description = {
    "AtB": "Agriculture, Hunting, Forestry, and Fishing",
    "C": "Mining and Quarrying",
    "D15t16": "Food, Beverages, and Tobacco",
    "D17t19": "Textiles, Textile Products, Leather, Footwear",
    "D21t22": "Pulp, Paper, Printing, Publishing",
    "D23": "Coke, Refined Petroleum, and Nuclear Fuel",
    "D24": "Chemicals and Chemical Products",
    "D25": "Rubber and Plastics",
    "D26": "Other Non-Metallic Mineral Products",
    "D27t28": "Basic and Fabricated Metals",
    "D29": "Machinery and Equipment n.e.c.",
    "D30t33": "Electrical and Optical Equipment",
    "D34t35": "Transport Equipment",
    "Dnec": "Manufacturing n.e.c.; Recycling",
    "E": "Electricity, Gas, and Water Supply",
    "F": "Construction",
    "G": "Wholesale and Retail Trade; Repairs",
    "H": "Hotels and Restaurants",
    "I60t63": "Transport and Storage",
    "I64": "Post and Telecommunications",
    "J": "Financial Intermediation",
    "K": "Real Estate, Renting, and Business Activities",
    "LtQ": "Public Administration, Education, Health, and Other Services",
    "xCONS_h": "Final Consumption Expenditure by Households",
    "xCONS_g": "Final Consumption Expenditure by Government",
    "xGFCF": "Gross Fixed Capital Formation",
    "xINV": "Changes in Inventories"
}

df["Sector"] = df["Sector"].map(code_to_description)

for (k,v) in code_to_description.items():
    print(k, " --- ", v)

# print(df["row_isic3"].value_counts(dropna=False))




AtB  ---  Agriculture, Hunting, Forestry, and Fishing
C  ---  Mining and Quarrying
D15t16  ---  Food, Beverages, and Tobacco
D17t19  ---  Textiles, Textile Products, Leather, Footwear
D21t22  ---  Pulp, Paper, Printing, Publishing
D23  ---  Coke, Refined Petroleum, and Nuclear Fuel
D24  ---  Chemicals and Chemical Products
D25  ---  Rubber and Plastics
D26  ---  Other Non-Metallic Mineral Products
D27t28  ---  Basic and Fabricated Metals
D29  ---  Machinery and Equipment n.e.c.
D30t33  ---  Electrical and Optical Equipment
D34t35  ---  Transport Equipment
Dnec  ---  Manufacturing n.e.c.; Recycling
E  ---  Electricity, Gas, and Water Supply
F  ---  Construction
G  ---  Wholesale and Retail Trade; Repairs
H  ---  Hotels and Restaurants
I60t63  ---  Transport and Storage
I64  ---  Post and Telecommunications
J  ---  Financial Intermediation
K  ---  Real Estate, Renting, and Business Activities
LtQ  ---  Public Administration, Education, Health, and Other Services
xCONS_h  ---  Final Consu

In [ ]:
countries = sorted([c for c in df['row_country'].unique() if c not in ['xROW', 'xTOT']])

region_map = {
    'USA': 'Americas',
    'CAN': 'Americas',
    'MEX': 'Americas',
    'AUT': 'Europe',
    'BEL': 'Europe',
    'DEU': 'Europe',
    'DNK': 'Europe',
    'ESP': 'Europe',
    'FIN': 'Europe',
    'FRA': 'Europe',
    'GBR': 'Europe',
    'GRC': 'Europe',
    'IRL': 'Europe',
    'ITA': 'Europe',
    'NLD': 'Europe',
    'PRT': 'Europe',
    'SWE': 'Europe',
    'AUS': 'Asia-Pacific',
    'CHN': 'Asia-Pacific',
    'HKG': 'Asia-Pacific',
    'IND': 'Asia-Pacific',
    'JPN': 'Asia-Pacific',
    'KOR': 'Asia-Pacific',
    'TWN': 'Asia-Pacific',
    'BRA': 'Americas'
}

#dict to store each df
country_dfs = {}

for country in countries:
    # Columns that belong to this country
    cols = ['row_country', 'Year', 'Sector'] + [c for c in df.columns if country in c]

    # Subset the dataframe for this country's rows
    country_df = df[df['row_country'] == country][cols].copy()

    rename_dict = {f"{country}_{k}": v for k, v in code_to_description.items() if f"{country}_{k}" in country_df.columns}
    country_df.rename(columns=rename_dict, inplace=True)

    #redundant info so drop row_country
    country_df.drop(columns=['row_country'], inplace=True)

    country_dfs[country] = country_df

print(f"Generated dataframes for {len(country_dfs)} countries: {list(country_dfs.keys())}")

#Lawrence Part
# plot y trends from year to year for each country, find L matrix and plot trends for all countries year to year for each entry
output_components = ["Final Consumption Expenditure by Households",
                     "Final Consumption Expenditure by Government",
                     "Gross Fixed Capital Formation",
                     "Changes in Inventories"]
df_all = []

for country, df_country in country_dfs.items():
    temp = df_country.copy()
    temp["Country"] = country
    temp["Region"] = country  # temporarily fill
    temp["Region"] = temp["Country"].map(region_map)  # map to region
    df_all.append(temp)

# Combine all countries
df_all = pd.concat(df_all, ignore_index=True)

component_titles = {
    "Final Consumption Expenditure by Households": "Final Consumption\nby Households",
    "Final Consumption Expenditure by Government": "Final Consumption\nby Government",
    "Gross Fixed Capital Formation": "Gross Fixed\nCapital Formation",
    "Changes in Inventories": "Changes in Inventories"
}

components_long = df_all.melt(
    id_vars=["Year", "Country"],
    value_vars=output_components,
    var_name="Component",
    value_name="Value"
)



Generated dataframes for 25 countries: ['AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHN', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'NLD', 'PRT', 'SWE', 'TWN', 'USA']


In [ ]:
sector_cols = [c for c in df_all.columns if c not in output_components + ["Year", "Country", "Region"]]

# If you have melted sector data with a 'Sector' column
sectors_long = df_all.melt(
    id_vars=["Year", "Country", "Region"],
    value_vars=[c for c in df_all.columns if c not in output_components + ["Year", "Country", "Region"]],
    var_name="Sector",
    value_name="Value"
)

# Drop any rows where Sector = "Sector"
sectors_long = sectors_long[sectors_long["Sector"] != "Sector"]


# Compute mean by Year, Region, Sector
regional_avg = sectors_long.groupby(["Year", "Region", "Sector"])["Value"].mean().reset_index()

#regional_avg['Sector'].unique()

In [ ]:
regional_avg

,Year,Region,Sector,Value
0,1965,Americas,"Agriculture, Hunting, Forestry, and Fishing",295.467282
1,1965,Americas,Basic and Fabricated Metals,496.931524
2,1965,Americas,Chemicals and Chemical Products,249.974153
3,1965,Americas,"Coke, Refined Petroleum, and Nuclear Fuel",184.675027
4,1965,Americas,Construction,553.626611
...,...,...,...,...
2479,2000,Europe,Rubber and Plastics,248.885135
2480,2000,Europe,"Textiles, Textile Products, Leather, Footwear",381.039803
2481,2000,Europe,Transport Equipment,1048.868243
2482,2000,Europe,Transport and Storage,1231.608852


In [ ]:
# #plotting cell
# sns.set_theme(style="whitegrid", context="talk")

# fig, axes = plt.subplots(1, 5, figsize=(26, 10), gridspec_kw={'width_ratios': [1,1,1,1,0.3]})

# for ax, comp in zip(axes[:4], output_components):
#     if comp not in components_long["Component"].unique():
#         ax.set_visible(False)
#         continue

#     sns.lineplot(
#         data=components_long[components_long["Component"] == comp],
#         x="Year", y="Value", hue="Country",
#         ax=ax,
#         ci=None
#     )
#     ax.set_title(component_titles.get(comp, comp))
#     ax.set_xlabel("Year")
#     ax.set_ylabel("Final Demand (Value)")
#     ax.get_legend().remove()

# # Legend-only axis
# axes[-1].axis("off")  # hide the last subplot’s frame
# handles, labels = axes[0].get_legend_handles_labels()
# axes[-1].legend(
#     handles, labels,
#     title="Country",
#     loc="center",
#     fontsize=12,
#     title_fontsize=16
# )

# plt.tight_layout()




In [ ]:


# sns.set_theme(style="whitegrid", context="talk")

# sectors = sorted(regional_avg["Sector"].unique())
# n_sectors = len(sectors)

# # Decide grid size
# ncols = 5
# nrows = math.ceil(n_sectors / ncols)

# fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 5*nrows))
# axes = axes.flatten()  # flatten for easy indexing

# for i, sector in enumerate(sectors):
#     ax = axes[i]
#     sns.lineplot(
#         data=regional_avg[regional_avg["Sector"] == sector],
#         x="Year", y="Value", hue="Region",
#         ax=ax,
#         ci=None
#     )
#     ax.set_title(sector, fontsize=10)
#     ax.set_xlabel("Year")
#     ax.set_ylabel("Avg Output")
#     ax.get_legend().remove()  # remove individual legends

# # Remove any empty subplots
# for j in range(i+1, len(axes)):
#     axes[j].axis("off")

# # Create a single legend
# handles, labels = axes[0].get_legend_handles_labels()
# fig.legend(
#     handles, labels,
#     title="Region",
#     loc="upper center",
#     bbox_to_anchor=(0.5, 1.05),
#     ncol=len(labels),
#     fontsize=12,
#     title_fontsize=14
# )

# plt.tight_layout()
# plt.show()


In [ ]:
# slighlty altered version that works with others in pipeline

def compute_L(country:str, year:int, output_components:list):
  #x=Ax+y
  string = country.upper()
  subset_country = df_all[df_all['Country']==string]
  subset_year = subset_country[subset_country['Year']==year]
  sector_names = subset_year['Sector'].to_list()
  y = subset_year[output_components].sum(axis=1).to_numpy()
  #subset_year.drop(output_components, axis=1, inplace = True)
  Z = subset_year.drop(columns=output_components + ['Year', 'Sector', 'Country', 'Region']).to_numpy() #intermediate matrix with the sectors interactions
  x = Z.sum(axis=0)+y #total demand which is intermediate demands of sectors + final demands of consumption
  A = Z/x[np.newaxis, :]
  I = np.eye(A.shape[0])
  L_matrix = np.linalg.inv(I-A)
  L_labeled = pd.DataFrame(L_matrix, index=sector_names, columns=sector_names)

  #added these to return year, country useful for plotting later
  return L_matrix, L_labeled, year, country_df

In [ ]:
# L_matrix_dict = {}
# years = sorted(df_all['Year'].unique())
# countries = sorted(df_all['Country'])
# output_components = ["Final Consumption Expenditure by Households",
#                      "Final Consumption Expenditure by Government",
#                      "Gross Fixed Capital Formation",
#                      "Changes in Inventories"]
# for country in countries:
#   L_matrix_dict[country]={}

#   for year in years:
#     try:
#       L_matrix, L_labeled, year, country_df = compute_L(country=country, year = year, output_components=output_components)
#       L_matrix_dict[country][year] = {'matrix':L_matrix,
#                                       'labeled':L_labeled}
#     except Exception as e:
#       print(f'Error for {country}, {year}: {e}')



In [ ]:
# import pickle
# with open("/content/drive/My Drive/CS439 Final Project/L_matrix_dict.pkl", 'wb') as f:
#   pickle.dump(L_matrix_dict, f)

In [ ]:
# df_countries = df_all[['Country', 'Region']].drop_duplicates()

# # Count countries per region
# region_counts = df_countries['Region'].value_counts().reset_index()
# region_counts.columns = ['Region', 'Count']

# # Plot
# sns.set_theme(style="whitegrid")
# plt.figure(figsize=(8,5))
# sns.barplot(data=region_counts, x='Region', y='Count', palette='Set2')
# plt.title("Number of Countries per Region")
# plt.xlabel("Region")
# plt.ylabel("Number of Countries")

In [ ]:
# sample_L, sample_labeled, year, country = compute_L('usa', 1970, output_components)
# sample_L.shape
# sample_L

In [ ]:
# from sklearn.decomposition import PCA
# #from sklearn.neighbors import KNeighborsClassifier
# import numpy as np
# import matplotlib.pyplot as plt
# def visualize_year(year, n_neighbors=3):
#     """
#     PCA + KNN classification plotted for a single year.
#     Uses L_matrix_dict[country][year]['matrix'].
#     """
#     # -------- Collect data for this year --------
#     X = []
#     y_region = []
#     y_country = []

#     for country in L_matrix_dict.keys():
#         if year not in L_matrix_dict[country]:
#             continue

#         mat = L_matrix_dict[country][year]['matrix']
#         X.append(mat.flatten())
#         y_region.append(region_map[country])
#         y_country.append(country)

#     X = np.array(X)

#     # -------- PCA for this year only --------
#     pca = PCA(n_components=2)
#     X_pca = pca.fit_transform(X)

#     # -------- Fit KNN in PCA space --------
#     knn = KNeighborsClassifier(n_neighbors=n_neighbors)
#     knn.fit(X_pca, y_region)

#     # -------- Make prediction (optional) --------
#     y_pred = knn.predict(X_pca)

#     # -------- Plot --------
#     plt.figure(figsize=(10, 7))

#     regions = sorted(set(y_region))
#     colors = {"Americas": "red", "Europe": "blue", "Asia-Pacific": "green"}

#     for region in regions:
#         idx = [i for i, r in enumerate(y_region) if r == region]
#         plt.scatter(
#             X_pca[idx, 0],
#             X_pca[idx, 1],
#             c=colors[region],
#             label=region,
#             s=120,
#             alpha=0.8,
#             edgecolor="black"
#         )

#     # Add labels
#     for i, country in enumerate(y_country):
#         plt.text(X_pca[i, 0] + 0.01, X_pca[i, 1] + 0.01,
#                  country, fontsize=9)

#     plt.title(f"PCA of Leontief Matrices — Year {year}")
#     plt.xlabel("PCA Component 1")
#     plt.ylabel("PCA Component 2")
#     plt.legend()
#     plt.grid(alpha=0.3)
#     plt.tight_layout()
#     plt.show()


In [ ]:
# from sklearn.decomposition import PCA

# def knn_data_pca(year, n_components=5):
#     X, y = knn_data(year)
#     pca = PCA(n_components=n_components)
#     X_pca = pca.fit_transform(X)
#     return X_pca, y
# X_pca, y = knn_data_pca(year, n_components=2)
# plt.figure(figsize=(8,6))
# sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=y)
# plt.title(f"PCA of L-matrices — Year {year}")
# plt.show()


In [ ]:
# import pickle
# #Computed all the L matrices and stored them, opening here
# with open("/content/drive/My Drive/CS439 Final Project/L_matrix_dict.pkl", 'rb') as f:
#     L_matrix_dict = pickle.load(f)


In [ ]:
# L_matrix_dict['USA'][1965]['labeled']

,"Agriculture, Hunting, Forestry, and Fishing",Mining and Quarrying,"Food, Beverages, and Tobacco","Textiles, Textile Products, Leather, Footwear","Pulp, Paper, Printing, Publishing","Coke, Refined Petroleum, and Nuclear Fuel",Chemicals and Chemical Products,Rubber and Plastics,Other Non-Metallic Mineral Products,Basic and Fabricated Metals,...,Manufacturing n.e.c.; Recycling,"Electricity, Gas, and Water Supply",Construction,Wholesale and Retail Trade; Repairs,Hotels and Restaurants,Transport and Storage,Post and Telecommunications,Financial Intermediation,"Real Estate, Renting, and Business Activities","Public Administration, Education, Health, and Other Services"
"Agriculture, Hunting, Forestry, and Fishing",1.421903,0.017020,0.375014,0.097597,0.068344,0.012374,0.040237,0.069896,0.025766,0.019398,...,0.096662,0.010644,0.017111,0.016398,0.122283,0.012391,0.007393,0.008533,0.007479,0.010641
Mining and Quarrying,0.043100,1.144711,0.021934,0.017563,0.031927,0.386505,0.069758,0.045630,0.203535,0.173986,...,0.040186,0.176158,0.042764,0.012718,0.010241,0.040000,0.016566,0.008862,0.010035,0.013648
"Food, Beverages, and Tobacco",0.251741,0.015673,1.230541,0.040121,0.037785,0.015343,0.055012,0.039533,0.026006,0.023046,...,0.032168,0.010090,0.012077,0.021410,0.308586,0.018018,0.007568,0.011156,0.008159,0.014452
"Textiles, Textile Products, Leather, Footwear",0.028671,0.020104,0.015179,1.561256,0.039732,0.012958,0.021801,0.147151,0.046714,0.037604,...,0.073502,0.012365,0.017394,0.014861,0.008629,0.017972,0.007574,0.005804,0.005644,0.011956
"Pulp, Paper, Printing, Publishing",0.052198,0.040327,0.063973,0.044579,1.504994,0.044028,0.109561,0.095746,0.141709,0.076963,...,0.068569,0.027733,0.034400,0.044690,0.024301,0.037617,0.025084,0.064166,0.028336,0.033027
"Coke, Refined Petroleum, and Nuclear Fuel",0.050668,0.026675,0.019217,0.014031,0.020189,1.064758,0.033771,0.030419,0.033132,0.029450,...,0.020332,0.025196,0.018172,0.011620,0.009007,0.058023,0.009870,0.007351,0.006939,0.010240
Chemicals and Chemical Products,0.145725,0.074526,0.057880,0.129175,0.115469,0.073620,1.380428,0.412259,0.140465,0.107368,...,0.092125,0.038673,0.043596,0.019355,0.024068,0.037030,0.018581,0.017013,0.019851,0.030565
Rubber and Plastics,0.020520,0.027172,0.013287,0.023590,0.028271,0.015539,0.032873,1.083958,0.045060,0.029287,...,0.043795,0.012386,0.016560,0.008061,0.005833,0.018793,0.007589,0.005194,0.008873,0.007849
Other Non-Metallic Mineral Products,0.014120,0.033293,0.016748,0.007746,0.010191,0.018892,0.024669,0.019551,1.266065,0.031267,...,0.022419,0.031511,0.071425,0.006209,0.006242,0.011817,0.020300,0.003880,0.007782,0.006599
Basic and Fabricated Metals,0.116549,0.300994,0.089532,0.055721,0.108895,0.154686,0.205758,0.158218,0.246743,2.185918,...,0.248841,0.147946,0.243134,0.039966,0.035231,0.121464,0.081506,0.029475,0.042314,0.052576
